<p><img height="45px" src="https://drive.google.com/uc?id=1toxeOL-eCjWBm2tkzaGOQT9LEw77PIi2"align="left" hspace="10px" vspace="0px"></p>

<h1>Conjuntos de datos desbalanceados</h1>
<br>

*Tiempo aproximado:* ***1 hora***

---

#**Introducción**

Usted trabaja como científico de datos para una empresa de telecomunicaciones, y su objetivo actual es predecir si un cliente abandonará el operador. Para alcanzar el objetivo, Usted cuenta con un *dataset* relacionado con la rotación de clientes. Este conjunto de datos tiene varias variables que describen el nivel de uso de una conexión móvil: duración total de llamadas en minutos, cargos de llamadas, llamadas realizadas durante ciertos períodos del día, detalles de llamadas internacionales y detalles de llamadas a servicio al cliente.
<br><br>
Este conjunto de datos es muy desbalanceado, y los casos en los que los clientes abandonan son la minoría. Entonces, antes de ajustar un clasificador para analizar el abandono de los clientes, Usted ha decidido corregir el desequilibrio de clases. Para esto, va a aplicar diferentes procesos de balanceo y los comparará para encontrar el mejor método antes de ajustar el modelo. 

# **Adquisición de datos y tratamiento inicial**

El conjunto de datos es un archivo CSV que está disponible en: https://raw.githubusercontent.com/lvmeninnovations/datasets/main/crispdm/churn.csv. 
<br><br>
Para iniciar, cargue los datos siguiendo el mismo procedimiento que ha hecho en prácticas anteriores.

In [45]:
# Importar la librería pandas
import pandas as pd

# Leer el conjunto de datos por medio de la URL proporcionada y asignarlo a la variable "churnData"
path = "https://raw.githubusercontent.com/lvmeninnovations/datasets/main/crispdm/churn.csv"
datosAbandono = pd.read_csv(path)

# Observar los primeros registros del dataset
datosAbandono.head()

,churn,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,No,128,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,No,107,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,No,137,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,No,84,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,No,75,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


Veamos los tipos de datos que tiene el *dataset*.

In [13]:
datosAbandono.dtypes

churn                          object
accountlength                   int64
internationalplan              object
voicemailplan                  object
numbervmailmessages             int64
totaldayminutes               float64
totaldaycalls                   int64
totaldaycharge                float64
totaleveminutes               float64
totalevecalls                   int64
totalevecharge                float64
totalnightminutes             float64
totalnightcalls                 int64
totalnightcharge              float64
totalintlminutes              float64
totalintlcalls                  int64
totalintlcharge               float64
numbercustomerservicecalls      int64
dtype: object

* **Normalización**

Como el conjunto de datos contiene valores numéricos, primero debemos normalizar los datos. El propósito de la **normalización** es hacer que todos los atributos numéricos estén en una escala común. Un método efectivo que podemos usar aquí para la función de normalización se llama ```MinMaxScaler()```, que convierte todos los datos numéricos a un rango escalado que podemos determinar. La función ```MinMaxScaler``` está disponible dentro del método de ```preprocessing``` de ```sklearn```:

In [14]:
# Método preprocessing de la librería sklearn
from sklearn import preprocessing

# Obtener un objeto MinMaxScaler para poder normalizar los datos
minmaxScaler = preprocessing.MinMaxScaler()

# Normalizar cada columna numérica (con el rango -1,1)
datosAbandono['alScaled'] = minmaxScaler.fit_transform(datosAbandono['accountlength'].values.reshape(-1,1))
datosAbandono['nvmmScaled'] = minmaxScaler.fit_transform(datosAbandono['numbervmailmessages'].values.reshape(-1,1))
datosAbandono['tdmScaled'] = minmaxScaler.fit_transform(datosAbandono['totaldayminutes'].values.reshape(-1,1))
datosAbandono['tdcScaled'] = minmaxScaler.fit_transform(datosAbandono['totaldaycalls'].values.reshape(-1,1))
datosAbandono['tdchScaled'] = minmaxScaler.fit_transform(datosAbandono['totaldaycharge'].values.reshape(-1,1))
datosAbandono['temScaled'] = minmaxScaler.fit_transform(datosAbandono['totaleveminutes'].values.reshape(-1,1))
datosAbandono['tecScaled'] = minmaxScaler.fit_transform(datosAbandono['totalevecalls'].values.reshape(-1,1))
datosAbandono['techScaled'] = minmaxScaler.fit_transform(datosAbandono['totalevecharge'].values.reshape(-1,1))
datosAbandono['tnmScaled'] = minmaxScaler.fit_transform(datosAbandono['totalnightminutes'].values.reshape(-1,1))
datosAbandono['tncScaled'] = minmaxScaler.fit_transform(datosAbandono['totalnightcalls'].values.reshape(-1,1))
datosAbandono['tnchScaled'] = minmaxScaler.fit_transform(datosAbandono['totalnightcharge'].values.reshape(-1,1))
datosAbandono['timScaled'] = minmaxScaler.fit_transform(datosAbandono['totalintlminutes'].values.reshape(-1,1))
datosAbandono['ticScaled'] = minmaxScaler.fit_transform(datosAbandono['totalintlcalls'].values.reshape(-1,1))
datosAbandono['tichScaled'] = minmaxScaler.fit_transform(datosAbandono['totalintlcharge'].values.reshape(-1,1))
datosAbandono['ncscScaled'] = minmaxScaler.fit_transform(datosAbandono['numbercustomerservicecalls'].values.reshape(-1,1))

Dado que hemos guardado los atributos numéricos transformados como variables separadas, podemos eliminar los atributos originales:

In [15]:
# Eliminar las columnas originales
datosAbandono.drop(['accountlength','numbervmailmessages',\
                'totaldayminutes','totaldaycalls',\
                'totaldaycharge','totaleveminutes',\
                'totalevecalls','totalevecharge',\
                'totalnightminutes','totalnightcalls',\
                'totalnightcharge','totalintlminutes',\
                'totalintlcalls','totalintlcharge',\
                'numbercustomerservicecalls'],\
               axis=1, inplace=True)

# Imprimir los primeros registros del conjunto de datos
datosAbandono.head()

,churn,internationalplan,voicemailplan,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled
0,No,no,yes,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111
1,No,no,yes,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111
2,No,no,no,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000
3,No,yes,no,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222
4,No,yes,no,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333


* **Transformar variables nominales a numéricas**

Crearemos las variables ficticias para todos los atributos nominales:

In [17]:
# Convertir todas las variables categóricas a variables dummy (excepto la clase)
datosAbandonoCat = pd.get_dummies(datosAbandono[['internationalplan','voicemailplan']])

# Visualizar el dataframe que contiene las variables dummy
datosAbandonoCat.head()

,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes
0,1,0,0,1
1,1,0,0,1
2,1,0,1,0
3,0,1,1,0
4,0,1,1,0


Una vez que se transforman los valores categóricos, deben combinarse con los valores numéricos escalados del *data frame*.

# **Creación de la versión de entrenamiento/prueba del dataset**

Vamos a separar los datos numéricos transformados del conjunto de datos original para luego concatenarlos con las variables categóricas ficticias que creamos en el paso anterior.

In [19]:
# Separamos los datos numéricos en otro dataframe (denominado churnNum)
datosAbandonoNum = datosAbandono[['alScaled','nvmmScaled',\
                      'tdmScaled','tdcScaled',\
                      'tdchScaled','temScaled',\
                      'tecScaled','techScaled',\
                      'tnmScaled','tncScaled',\
                      'tnchScaled','timScaled',\
                      'ticScaled','tichScaled','ncscScaled']]

# Vemos el tamaño del dataframe que contiene los datos numéricos
datosAbandonoNum.shape

(5000, 15)

En este paso, concatenamos las variables categóricas transformadas y las variables numéricas usando la función ```pd.concat()``` para formar la variable ```X``` (contiene las variables independientes). La etiqueta de la variable de destino se almacena como la variable ```Y``` (variable dependiente o clase)



In [20]:
# Fusión de los dataframe

# Preparar las variables X (el parámetro axis=1 indica que la concatenación se hace a lo ancho)
X = pd.concat([datosAbandonoCat, datosAbandonoNum], axis=1) 
print(X.shape)

# Preparar la variable Y
Y = datosAbandono['churn']
print(Y.shape)
X.head()

(5000, 19)
(5000,)


,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled
0,1,0,0,1,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111
1,1,0,0,1,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111
2,1,0,1,0,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000
3,0,1,1,0,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222
4,0,1,1,0,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333


Ahora, necesitaremos la función ```train_test_split()```, por lo que las importaremos desde ```sklearn```. Esta función nos permitirá dividir el *dataset* en un conjunto de datos de entrenamiento y otro de prueba.
<br><br>
Dividiremos los datos entre datos de entrenamiento y de prueba en una proporción de **70:30** utilizando la variable ```test_size=0.3``` en la función de división. También establecemos un valor para el parámetro ```random_state``` para la reproducibilidad del código:

In [21]:
# Importar la función train_test_split
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

# **Creación de un modelo de clasificación sin realizar balancedo de clases**

Antes de aplicar los métodos de balanceo, veamos cómo se comporta un modelo de clasificación (en este caso de Regresión Logística) al ser entrenado con los datos desbalanceados.
<br><br>
Entrene el algoritmo de Regresión Logística usando la función ```.fit``` en los datos de entrenamiento.

In [22]:
from sklearn.linear_model import LogisticRegression

# Definir la función LogisticRegression y entrenar con los datos de entrenamiento
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Ahora que el modelo se ha creado, realizaremos predicciones con el conjunto de prueba y generaremos las métricas de rendimiento. Veamos primero la precisión del modelo (*accuracy*) y la matriz de confusión.

In [23]:
# Importar la función confusion_matrix de sklearn
from sklearn.metrics import confusion_matrix

# Hacemos predicciones con el conjunto de prueba
predicciones = modelo.predict(X_test)

# Obtenemos la matriz de confusión a partir de los resultados obtenidos
confusionMatrix = confusion_matrix(y_test, predicciones)
print("---MATRIZ DE CONFUSIÓN---")
print(confusionMatrix)
print("-------------------------")

# Obtenemos la precisión del modelo
accuracy = modelo.score(X_test, y_test)
print('La precisión del modelo de Regresión Logística con en el conjunto de prueba es:', accuracy)


---MATRIZ DE CONFUSIÓN---
[[1259   24]
 [ 179   38]]
-------------------------
La precisión del modelo de Regresión Logística con en el conjunto de prueba es: 0.8646666666666667


## Ejercicio 1

```sklearn``` le permite utilizar la función ```classification_report``` para obtener un reporte más completo del rendimiento del modelo de clasificacion. **Importe dicha función y obtenga el reporte de su modelo**. El uso de esta función es similar al procedimiento que se hizo con la función ```confusion_matrix``` y hacen parte del mismo paquete de ```sklearn```.

In [25]:
# Escriba su código a continuación y presione Shift + Enter para ejecutar


**SOLUCIÓN DEL EJERCICIO:**

Haga doble clic **aquí** para ver la solución del Ejercicio 1. 

<!-- La respuesta es la siguiente:

# Importar la función classification_report de sklearn
from sklearn.metrics import classification_report
print("---REPORTE DE LA CLASIFICACIÓN---")
reporte = classification_report(y_test, predicciones)
print(reporte)

-->

Como se puede observar, aunque la precisión general (*accuracy*) del modelo es buena, la matriz de confusión nos indica que esta medida es engañosa, pues es evidente que no se ha obtenido un buen resultado para la clase minoritaria. El reporte que obtuvo en el ejercicio anterior también le demuestra el mal comportamiento (en este caso, vea especialmente el valor de *Recall*). El desbalanceo de clase es una de las principales razones detrás de las malas métricas que hemos obtenido con el clasificador de Regresión Logística.

# **Estrategias para lidiar con el desbalanceo de clases**

En muchas circunstancias, recopilar más datos, especialmente de las clases minoritarias, puede ser un desafío. En tales circunstancias, debemos adoptar diferentes estrategias para trabajar con nuestras limitaciones y esforzarnos por balancear nuestro conjunto de datos.

## *Método de Remuestreo (Submuestreo o Undersampling)*

La idea detrás del remuestreo es elegir al azar muestras de la clase mayoritaria para hacer que el conjunto de datos final sea más equilibrado. Como resultado, la clase minoritaria tendrá el mismo número de instancias originales, mientras que la clase mayoritaria estará submuestreada. El remuestreo de este tipo se denomina **submuestreo aleatorio** porque estamos submuestreando la clase mayoritaria.
<br><br>
Antes de continuar con el proceso de remuestreo, concatenamos los conjuntos de datos ```X_train``` y ```y_train``` en un solo conjunto de datos (esto es el conjunto de datos de entrenamiento con sus respectivas etiquetas). Esto se hace para facilitar el proceso de remuestreo en los pasos siguientes. 

In [46]:
# Concatenar train_x train_y para facilitar las operaciones posteriores
trainData = pd.concat([X_train,y_train],axis=1)
trainData.head()

,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
4036,1,0,0,1,0.256198,0.500000,0.609388,0.484848,0.609270,0.695628,0.894118,0.695891,0.395949,0.622857,0.396173,0.515,0.10,0.514815,0.222222,No
2883,1,0,1,0,0.504132,0.000000,0.595733,0.296970,0.595716,0.652736,0.688235,0.652863,0.605570,0.560000,0.605515,0.490,0.55,0.490741,0.111111,No
4162,0,1,1,0,0.012397,0.000000,0.482788,0.581818,0.482764,0.362387,0.552941,0.362342,0.620506,0.491429,0.620709,0.710,0.20,0.709259,0.000000,Yes
4640,1,0,1,0,0.450413,0.000000,0.714936,0.551515,0.714859,0.569700,0.558824,0.569719,0.630380,0.400000,0.630838,0.645,0.10,0.644444,0.111111,Yes
2430,1,0,0,1,0.491736,0.769231,0.364438,0.600000,0.364458,0.681056,0.458824,0.681009,0.505570,0.691429,0.505909,0.780,0.15,0.779630,0.000000,No


Para hacer el proceso de submuestreo, primero separaremos las instancias de la clase minoritaria y de la clase mayoritaria. Esto es necesario porque tenemos que muestrear por separado a la clase mayoritaria para crear un conjunto de datos equilibrado.

* **Obtener las instancias de la clase minoritaria**

Para separar la clase minoritaria, tenemos que identificar los índices del conjunto de datos donde las instancias tienen la etiqueta 'yes' en su clase. 
<br><br>
Los índices se identifican mediante la función ```.index()```:


In [27]:
# Encontrar los índices de las instancias donde la etiqueta/clase (columna churn) es 'yes'
minIndices = trainData[trainData['churn']=='Yes'].index

# Imprimir el número de instancias con la etiqueta 'yes'
print(len(minIndices))

490


Una vez que se identifican los índices de la clase minoritaria, se separan del conjunto de datos principal mediante la función ```.loc()``` y se almacenan en una nueva variable para la clase minoritaria.

In [28]:
# Obtener las instancias que representan la clase minoritara
minData = trainData.loc[minIndices]

# Imprimir el tamaño (el número de filas debe coincidir con el tamaño que arrojó código anterior)
print(minData.shape)

(490, 20)


* **Obtener las instancias de la clase mayoritaria**

Para obtener la instancias de la clase mayoritaria, realice el Ejercicio 2

### Ejercicio 2

**Con base en el proceso anterior, separe la clase mayoritaria del conjunto de datos principal y almacene el resultado en la variable ```majData```.**

In [31]:
# Escriba su código a continuación y presione Shift + Enter para ejecutar
# Encontrar los índices de las instancias donde la etiqueta/clase (columna churn) es 'no'
majIndices = trainData[trainData['churn']=='No'].index

# Obtener las instancias que representan la clase minoritara
majData = trainData.loc[majIndices]

**SOLUCIÓN DEL EJERCICIO:**

Haga doble clic **aquí** para ver la solución del Ejercicio 2. 

<!-- La respuesta es la siguiente:



-->

* **Muestreo de la clase mayoritaria**

Tome una muestra aleatoria de la clase mayoritaria con una longitud igual a la de la clase minoritaria, para poder equilibrar el conjunto de datos.

In [32]:
# Tomar aleatoriamente instancias de la clase mayoritaria
# el número de instancias será igual al número de instancias de la clase minoritaria = len(minIndices)
majMuestra = majData.sample(n=len(minIndices), random_state = 123)

# Imprimir el tamaño de la muestra
print(majMuestra.shape)

# Imprimir los primeros registros dela muestra
majMuestra.head()

(490, 20)


,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
1807,1,0,1,0,0.450413,0.000000,0.557895,0.624242,0.557898,0.549079,0.723529,0.549013,0.344051,0.405714,0.344401,0.645,0.05,0.644444,0.333333,No
4578,1,0,1,0,0.475207,0.000000,0.244097,0.533333,0.244143,0.318394,0.658824,0.318344,0.495949,0.520000,0.496342,0.550,0.10,0.550000,0.111111,No
355,1,0,1,0,0.123967,0.000000,0.472546,0.636364,0.472557,0.218037,0.547059,0.218052,0.541013,0.560000,0.541362,0.635,0.10,0.635185,0.111111,No
23,1,0,1,0,0.454545,0.000000,0.314083,0.624242,0.314090,0.377509,0.600000,0.377548,0.480000,0.600000,0.480023,0.385,0.30,0.385185,0.222222,No
1541,1,0,0,1,0.194215,0.692308,0.656899,0.557576,0.656794,0.460819,0.711765,0.461016,0.683544,0.497143,0.683737,0.380,0.20,0.379630,0.333333,No


* **Prepare el nuevo dataset**

Ahora podemos concatenar estos conjuntos de datos individuales (conjunto de la clase minoritaria y muestreo de la clase mayoritaria) usando la función ```pd.concat()```. En este caso, estamos concatenando verticalmente los datos, por lo tanto, se usa el parámetro ```axis = 0```.

In [33]:
# Concatenar ambos datasets
balancedData = pd.concat([minData,majMuestra],axis = 0)
print('Tamaño del conjunto de datos balanceado',balancedData.shape)

Tamaño del conjunto de datos balanceado (980, 20)


Ahora, baraje el nuevo conjunto de datos para que tanto las clases minoritarias como las mayoritarias se distribuyan uniformemente. Para esto, utilice función ```shuffle()```:

In [34]:
# Importar la función shuffle
from sklearn.utils import shuffle

# Barajar el dataset
balancedData = shuffle(balancedData)
# Imprimir los primeros registros del dataset
balancedData.head()

,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
4120,0,1,1,0,0.128099,0.000000,0.755334,0.454545,0.755355,0.329117,0.452941,0.329020,0.430127,0.731429,0.430501,0.740,0.30,0.740741,0.222222,Yes
2862,1,0,0,1,0.516529,0.461538,0.167568,0.757576,0.167503,0.839978,0.529412,0.840181,0.402278,0.417143,0.402364,0.605,0.30,0.605556,0.000000,No
4576,1,0,1,0,0.537190,0.000000,0.730014,0.581818,0.729920,0.619192,0.664706,0.619217,0.481519,0.520000,0.481711,0.555,0.15,0.555556,0.111111,Yes
2203,0,1,1,0,0.396694,0.000000,0.412518,0.624242,0.412483,0.809183,0.547059,0.809447,0.607089,0.685714,0.607203,0.550,0.10,0.550000,0.444444,Yes
3851,1,0,0,1,0.731405,0.615385,0.423044,0.575758,0.423025,0.546054,0.323529,0.546102,0.473418,0.645714,0.473270,0.520,0.35,0.520370,0.666667,Yes


Ahora, separe el conjunto de datos barajado en variables independientes, ```X_train_sub```, y variables dependientes, ```y_train_sub```. La separación de las variables independientes la podemos realizar utilizando la función ```.iloc()``` de pandas. (Utilizamos el sufijo ```_sub``` en los nombres de las variables para indicar que representan el conjunto de datos de entrenamiento obtenido por medio de un proceso de submuestreo).

In [35]:
# Crear los nuevos X_train y y_train

# Crear las nuevas variables independientes
X_train_sub = balancedData.iloc[:,0:19]
print('Tamaño del nuevo conjunto de entrenamiento (variables independientes)', X_train_sub.shape)

# Crear la nueva variable dependiente y_train
y_train_sub = balancedData['churn']
print('Tamaño del nuevo conjunto de entrenamiento (varible dependiente)',y_train_sub.shape)

Tamaño del nuevo conjunto de entrenamiento (variables independientes) (980, 19)
Tamaño del nuevo conjunto de entrenamiento (varible dependiente) (980,)


Nuestro nuevo dataset de entrenamiento está listo para ajustar un nuevo modelo de Regresión Logística (para este caso particular) y comparar los resultados con el comportamiento del modelo construido sin balancear los datos. Pero antes de realizar esta evaluación, realicemos otros procedimientos de balanceo de datos.


## *Método de sobremuestreo (SMOTE)*

Generaremos muestras sintéticas de la clase minoritaria utilizando el algoritmo **SMOTE** y luego equilibraremos el conjunto de datos. 
<br><br>
* **Importar las librerías necesarias:**

Las librerías necesarias para sobremuestrear el conjunto de entrenamiento son ```smote_variants``` y ```numpy```.

*NOTA: debemos instalar la librería ```smote_variants``` antes de importarla.*

In [36]:
!pip install smote-variants		

import smote_variants as sv
import numpy as np

     |████████████████████████████████| 143kB 3.8MB/s 
  Created wheel for minisom: filename=MiniSom-2.2.9-cp37-none-any.whl size=8603 sha256=214dc41ca3b2eca0ecbd939832258f6b533b0fcc46811de08992f7cf60632cbe
  Stored in directory: /root/.cache/pip/wheels/de/a0/08/5234d6b02b29c561f62b6c985e2eb7d480fb0b92359a8c74e4
  Created wheel for statistics: filename=statistics-1.0.3.5-cp37-none-any.whl size=7454 sha256=352b94eb1bc6e1f101969b060c18288769ebf3943f3ab35ee99701034a2df484
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built minisom statistics


* **Realizar el sobremuestreo de la clase minoritaria**

Ahora, cree una instancia de la librería SMOTE en una variable llamada ```oversampler``` usando la función ```sv.SMOTE()```. Esta es una forma común de instanciar cualquiera de las variantes de SMOTE ```smote_variants```.
<br><br>
*NOTA: aquí puede ver la documentación de todas las variantes de SMOTE disponibles en la librería: https://smote-variants.readthedocs.io/en/latest/oversamplers.html*

In [37]:
# Instanciando la clase SMOTE 
oversampler= sv.SMOTE()

Utilice la función ```.sample()``` de ```oversampler``` para hacer el proceso de sobremuestreo. Tanto las variables ```X``` como las ```y``` se deben convertir en matrices numpy antes de aplicar la función ```.sample()```:

In [38]:
# Crear un nuevo conjunto de datos de entrenamiento
X_train_smote, y_train_smote = oversampler.sample(np.array(X_train), np.array(y_train))

2021-05-10 23:48:22,603:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


* **Observar el dataset obtenido**

Ahora, imprima loa tamaños de las nuevas variables ```X``` y ```y```, y el conteo de las clases. Observará que el tamaño del conjunto de datos general ha aumentado. Este aumento de tamaño se atribuye al hecho de que la clase minoritaria ha sido sobremuestreada de 490 a 3010:

In [39]:
# Tamaño después de hacer sobremuestreo
print('Después de hacer sobremuestreo, el tamaño de train_X:', X_train_smote.shape)
print('Después de hacer sobremuestreo, el tamaño de train_y:', y_train_smote.shape)
print("Después de hacer sobremuestreo, conteo de la etiqueta 'Yes':", sum(y_train_smote=='Yes'))
print("Después de hacer sobremuestreo, conteo de la etiqueta 'no':", sum(y_train_smote=='No'))

Después de hacer sobremuestreo, el tamaño de train_X: (6020, 19)
Después de hacer sobremuestreo, el tamaño de train_y: (6020,)
Después de hacer sobremuestreo, conteo de la etiqueta 'Yes': 3010
Después de hacer sobremuestreo, conteo de la etiqueta 'no': 3010


### Ejercicio 3

**Siguiendo los mismos pasos que con SMOTE, realice un proceso de sobremuestreo implementando MSMOTE.** Asígne a las variables ```X_train_msmote, y_train_msmote``` el conjunto de datos de entrenamiento resultante

In [40]:
# Escriba su código a continuación y presione Shift + Enter para ejecutar


**SOLUCIÓN DEL EJERCICIO:**

Haga doble clic **aquí** para ver la solución del Ejercicio 3. 

<!-- La respuesta es la siguiente:



-->

# **Comparación**

Hemos obtenido tres nuevas versiones del conjunto de entrenamiento. Vamos a crear un modelo de clasificación (de Regresión Logística para este caso particular) para cada una de ellas. De esta manera podremos comparar los resultados y elegir el método de balanceo más conveniente para nuestro caso particular.


In [42]:

# Instanciando la clase MSMOTE 
oversampler= sv.MSMOTE()

# Crear un nuevo conjunto de datos de entrenamiento
X_train_msmote, y_train_msmote = oversampler.sample(np.array(X_train), np.array(y_train))

# Tamaño después de hacer sobremuestreo
print('Después de hacer sobremuestreo, el tamaño de train_X:', X_train_msmote.shape)
print("Después de hacer sobremuestreo, conteo de la etiqueta 'Yes':", sum(y_train_msmote=='Yes'))
print("Después de hacer sobremuestreo, conteo de la etiqueta 'no':", sum(y_train_msmote=='No'))
# Creación y entrenamiento del modelo con base en el resultado de submuestreo
modelo1 = LogisticRegression()
modelo1.fit(X_train_sub, y_train_sub)

# Creación y entrenamiento del modelo con base en el resultado de sobremuestreo con SMOTE
modelo2 = LogisticRegression()
modelo2.fit(X_train_smote, y_train_smote)

# Creación y entrenamiento del modelo con base en el resultado de sobremuestreo con MSMOTE
modelo3 = LogisticRegression()
modelo3.fit(X_train_msmote, y_train_msmote)

2021-05-10 23:48:53,585:INFO:MSMOTE: Running sampling via ('MSMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


Después de hacer sobremuestreo, el tamaño de train_X: (6020, 19)
Después de hacer sobremuestreo, conteo de la etiqueta 'Yes': 3010
Después de hacer sobremuestreo, conteo de la etiqueta 'no': 3010


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Ahora, necesitamos hacer predicciones con el conjunto de datos de prueba ```X_test```, con cada uno de los modelos:

In [43]:
# Predicción utilizando SUBMUESTREO
pred_sub = modelo1.predict(X_test)

# Predicción utilizando SMOTE
pred_smote = modelo2.predict(X_test)

# Predicción utilizando MSMOTE
pred_msmote = modelo3.predict(X_test)

## Ejercicio 4

**Obtenga e imprima la matriz de confusión para cada modelo, así como los informes de clasificación.**

In [44]:
# Escriba su código a continuación y presione Shift + Enter para ejecutar

from sklearn.metrics import classification_report

# Obtenemos la matriz de confusión a partir de los resultados obtenidos
confusionMatrix1 = confusion_matrix(y_test, pred_sub)
confusionMatrix2 = confusion_matrix(y_test, pred_smote)
confusionMatrix3 = confusion_matrix(y_test, pred_msmote)

print("---RESULTADOS MODELO 1---")
print(confusionMatrix1)
reporte1 = classification_report(y_test, pred_sub)
print(reporte1)

print("---RESULTADOS MODELO 2---")
print(confusionMatrix2)
reporte2 = classification_report(y_test, pred_smote)
print(reporte2)

print("---RESULTADOS MODELO 3---")
print(confusionMatrix3)
reporte3 = classification_report(y_test, pred_msmote)
print(reporte3)


---RESULTADOS MODELO 1---
[[1032  251]
 [  57  160]]
              precision    recall  f1-score   support

          No       0.95      0.80      0.87      1283
         Yes       0.39      0.74      0.51       217

    accuracy                           0.79      1500
   macro avg       0.67      0.77      0.69      1500
weighted avg       0.87      0.79      0.82      1500

---RESULTADOS MODELO 2---
[[1008  275]
 [  55  162]]
              precision    recall  f1-score   support

          No       0.95      0.79      0.86      1283
         Yes       0.37      0.75      0.50       217

    accuracy                           0.78      1500
   macro avg       0.66      0.77      0.68      1500
weighted avg       0.86      0.78      0.81      1500

---RESULTADOS MODELO 3---
[[1042  241]
 [  55  162]]
              precision    recall  f1-score   support

          No       0.95      0.81      0.88      1283
         Yes       0.40      0.75      0.52       217

    accuracy           

**SOLUCIÓN DEL EJERCICIO:**

Haga doble clic **aquí** para ver la solución del Ejercicio 3. 

<!-- La respuesta es la siguiente:

from sklearn.metrics import classification_report

# Obtenemos la matriz de confusión a partir de los resultados obtenidos
confusionMatrix1 = confusion_matrix(y_test, pred_sub)
confusionMatrix2 = confusion_matrix(y_test, pred_smote)
confusionMatrix3 = confusion_matrix(y_test, pred_msmote)

print("---RESULTADOS MODELO 1---")
print(confusionMatrix1)
reporte1 = classification_report(y_test, pred_sub)
print(reporte1)

print("---RESULTADOS MODELO 2---")
print(confusionMatrix2)
reporte2 = classification_report(y_test, pred_smote)
print(reporte2)

print("---RESULTADOS MODELO 3---")
print(confusionMatrix3)
reporte3 = classification_report(y_test, pred_msmote)
print(reporte3)

-->

A partir del informe de clasificación, podemos ver que MSMOTE tiene la mejor precisión, 80%, en comparación con las técnicas SMOTE y de submuestreo, que alcanzan el 78% y el 79%, respectivamente. Sin embargo, sabemos que es importante tener en cuenta los valores de *Recall*, especialmente de la clase minoritaria.
<br><br>
Vemos que SMOTE y MSMOTE tienen el valor más alto (76%) para la métrica *Recall*. Esto significa que estos modelos han identificado correctamente al 76% de los clientes que probablemente abandonen el operador. El submuestreo aleatorio tiene un valor de *Recall* más bajo de 74%. De esta manera, tenemos una situación en la que MSMOTE tiene mejores resultados. 
<br><br>
También es importante mirar los puntajes de **f1**, que es un puntaje ponderado entre la *Precisión de clase* y *Recall*. De todos los puntajes f1, vemos que MSMOTE tiene el puntaje f1 más alto del 52%, con SMOTE 50% y con submuestreo aleatorio 51%. Por lo tanto, **podemos seleccionar MSMOTE como la mejor técnica para equilibrar en este contexto.

---

